In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
df = pd.read_csv('data/universitas_brawijaya_scholar_results2.csv')
abstracts = df['abstract'].dropna().tolist()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Download the necessary data for punkt
nltk.download('punkt_tab')

# Existing code remains the same
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

processed_abstracts = [preprocess(abstract) for abstract in abstracts]